In [28]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [29]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

## Get Indeks Link

In [42]:
links = getIndeksLink([], 1, 'news', 'news', '2018-07-28')

page  1
https://indeks.kompas.com/news/2018-07-28/1
page  2
https://indeks.kompas.com/news/2018-07-28/2
page  3
https://indeks.kompas.com/news/2018-07-28/3
page  4
https://indeks.kompas.com/news/2018-07-28/4
page  5
https://indeks.kompas.com/news/2018-07-28/5
page  6
https://indeks.kompas.com/news/2018-07-28/6
page  7
https://indeks.kompas.com/news/2018-07-28/7


In [43]:
links

[['http://regional.kompas.com/read/2018/07/28/23173601/1-agustus-api-obor-asian-games-2018-mampir-ke-danau-toba',
  'news'],
 ['http://nasional.kompas.com/read/2018/07/28/23103901/bertemu-demokrat-pks-akan-bahas-opsi-capres-cawapres-alternatif',
  'news'],
 ['http://regional.kompas.com/read/2018/07/28/23083031/karir-politik-bupati-lampung-selatan-sebelum-terjegal-fee-rp-700-juta',
  'news'],
 ['http://regional.kompas.com/read/2018/07/28/22524061/sebelum-ott-kpk-lampung-selatan-tiga-kali-dapat-penilaian-zona-merah',
  'news'],
 ['http://regional.kompas.com/read/2018/07/28/22505251/tilang-on-the-spot-segera-diterapkan-di-surabaya',
  'news'],
 ['http://regional.kompas.com/read/2018/07/28/22453671/di-karawang-aksi-long-march-laskar-santri-disambut-pawai-obor',
  'news'],
 ['http://regional.kompas.com/read/2018/07/28/22255841/sebelum-dikirim-ilegal-ke-malaysia-31-wna-bangladesh-disembunyikan-di-hutan',
  'news'],
 ['http://megapolitan.kompas.com/read/2018/07/28/22232341/syarif-hasil-pemeri

In [97]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="article__list clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
#         print(link)
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = el_page.findAll('div', class_='paging__item')[-1].find('a')
        if  el_page.findAll('div', class_='paging__item')[-1].find('a', class_="paging__link paging__link--active"):
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

In [116]:
articles = {}
url = "https://indeks.kompas.com/news/2018-07-28/7"
response = requests.get(url)
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
# soup

In [117]:
el_page = soup.find('div', class_="paging__wrap clearfix")
el_page

<div class="paging__wrap clearfix"><div class="paging__item"><a class="paging__link paging__link--next" data-ci-pagination-page="1" href="http://indeks.kompas.com/news/2018-07-28" rel="start">‹ First</a></div><div class="paging__item"><a class="paging__link paging__link--prev" data-ci-pagination-page="6" href="http://indeks.kompas.com/news/2018-07-28/6" rel="prev">Prev</a></div><div class="paging__item"><a class="paging__link" data-ci-pagination-page="5" href="http://indeks.kompas.com/news/2018-07-28/5">5</a></div><div class="paging__item"><a class="paging__link" data-ci-pagination-page="6" href="http://indeks.kompas.com/news/2018-07-28/6">6</a></div><div class="paging__item"><a class="paging__link paging__link--active" href="#">7</a></div></div>

In [118]:
a_page = el_page.findAll('div', class_='paging__item')[-1].find('a')
a_page

<a class="paging__link paging__link--active" href="#">7</a>

In [119]:
soup.findAll('div', class_='paging__item')[-1].find('a')

<a class="paging__link paging__link--active" href="#">7</a>

In [114]:
el_page.findAll('div', class_='paging__item')[-1].find('a', class_="paging__link paging__link--active")

In [115]:
max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
max_page

7

## Get Detail Berita

In [ ]:
def getDetailBerita(links):
    all_articles = {}
    for link in links:
        #link
        articles['url'] = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])

        #category
        articles['category'] = link[1]
        articles['url'] = url

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text

        #article
        article = soup.find("div", class_="read__content")

        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")

        #extract author
        articles['author'] = soup.find('div', class_="read__author").text

        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'kompas'

        #extract comments count
#         options = Options()
#         options.add_argument('--headless')
#         options.add_argument('--disable-gpu')  # Last I checked this was necessary.
#         options.add_argument('--disable-extensions')

#         driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
#         driver.get(url)
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html5lib')

#         comment = soup.find('div', class_="span4 comments-count tright")
#         comment_num = comment.text.replace('Ada ', '')
#         comment_num = comment_num.replace(' komentar untuk artikel ini', '')
        articles['comments'] = 0

        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        articles['tags'] = tags

        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']


        #hapus link sisip
        #         for link in detail.findAll('table', class_="linksisip"):
        #             link.decompose()

        #hapus video sisip
        #         for tag in detail.findAll('div', class_="detail_tag"):
        #             tag.decompose()

        #hapus all setelah clear fix
        #for det in detail.find('div', class_="wfull fl rl"):
        #     det.decompose()

        #hapus all script
        for script in article.findAll('script'):
            script.decompose()

        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        #articles['content'] = re.sub('google*','', content).strip(' ')
        articles['content']
        #print('memasukkan berita id ', articles['id'])

        all_articles.append(articles)

    return all_articles

In [66]:
url = "https://regional.kompas.com/read/2018/07/28/22524061/sebelum-ott-kpk-lampung-selatan-tiga-kali-dapat-penilaian-zona-merah"

In [67]:
articles = {}
url = "https://regional.kompas.com/read/2018/07/28/22524061/sebelum-ott-kpk-lampung-selatan-tiga-kali-dapat-penilaian-zona-merah"
response = requests.get(url)
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
# soup

In [ ]:
details = [getDetailBerita(links[0:2])]
details

In [ ]:
df = pd.DataFrame(detail)

In [445]:
df

,author,comments,content,id,images,pubdate,source,subcategory,tags,title,url
0,"Kontributor Lampung, Eni Muslihah",7,"BANDAR LAMPUNG, KOMPAS.com - Ombudsman RI Per...",1634180,https://asset.kompas.com/crop/0x0:1000x667/780...,2018-07-28 22:52:40,kompas,Regional,"KPK, Ombudsman, Bandar Lampung, Lampung Selata...","Sebelum OTT KPK, Lampung Selatan Tiga Kali Dap...",https://regional.kompas.com/read/2018/07/28/22...


In [446]:
soup.findAll('a', class_="inner-link")

[<a class="inner-link" href="http://indeks.kompas.com/tag/Bandar-Lampung" style="color:#428bca;" target="_blank">BANDAR LAMPUNG</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Ombudsman" style="color:#428bca;" target="_blank">Ombudsman</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Lampung-Selatan" style="color:#428bca;" target="_blank">Lampung Selatan</a>,
 <a class="inner-link" href="http://nasional.kompas.com/read/2018/07/28/18401441/kasus-suap-bupati-lampung-selatan-kpk-geledah-lima-tempat" target="_blank">Kasus Suap Bupati Lampung Selatan, KPK Geledah Lima Tempat</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/OTT" style="color:#428bca;" target="_blank">OTT</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/KPK" style="color:#428bca;" target="_blank">KPK</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Zainuddin-Hasan" style="color:#428bca;" target="_blank">Zainuddin Hasan</a>,
 <a class="inner-link" href="htt

In [438]:
# detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
# content = re.sub(r'\n|\t|\b|\r','',detail.text)
# articles['content'] = re.sub('google.*','', content).strip(' ')
# articles['content']
# # print('memasukkan berita id ', articles['id'])